In [1]:
# Importing the libraries
import transformers
import datasets
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Importing the transformers modules
from transformers import DistilBertTokenizer,TFDistilBertForSequenceClassification
from datasets import load_dataset

In [3]:
# Dataset loading

dataset = load_dataset("Arsive/toxicity_classification_jigsaw")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 25960
    })
    validation: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 6490
    })
    test: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        num_rows: 153164
    })
})

In [21]:
# Instantiating the tokenizer and model

checkpoint = "distilbert/distilbert-base-uncased"

tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = TFDistilBertForSequenceClassification.from_pretrained(checkpoint,num_labels=6)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [19]:
# Tokenization function

def tokenization(batch):
    return tokenizer(batch['comment_text'],padding=True,max_length=125,truncation=True,return_tensors="tf")

In [20]:
# Using the tokenizer function to tokenize our dataset

dataset = dataset.map(tokenization,batched=True)

Map: 100%|██████████| 153164/153164 [03:51<00:00, 661.68 examples/s]


In [60]:
# Prepare train and validation dataset for TensorFlow with multiple labels

def prepare_dataset(example):
    text = example["comment_text"]
    labels = {label: example[label] for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']}
    return {"comment_text": text, "labels": labels}

train_dataset = dataset["train"].map(prepare_dataset)
validation_dataset = dataset["validation"].map(prepare_dataset)

In [67]:
# Creating
train_dataset = tf.data.Dataset.from_tensor_slices((
    train_dataset["comment_text"],
    train_dataset["labels"]
)).shuffle(100).batch(8)

validation_dataset = tf.data.Dataset.from_tensor_slices((
    validation_dataset["comment_text"],
    validation_dataset["labels"]
)).batch(8)


ValueError: Attempt to convert a value ({'identity_hate': 1, 'insult': 1, 'obscene': 1, 'severe_toxic': 0, 'threat': 0, 'toxic': 1}) with an unsupported type (<class 'dict'>) to a Tensor.

In [61]:
# Shuffling the dataset and creating batches

train_dataset = train_dataset.shuffle(100).batch(8)
validation_dataset = validation_dataset.batch(8)

AttributeError: 'Dataset' object has no attribute 'batch'

[{'identity_hate': 1,
  'insult': 1,
  'obscene': 1,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 1},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 0,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 0,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 0,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 0,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 1,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 1,
  'obscene': 1,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 1},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 0,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 0,
  'severe_toxic': 0,
  'threat': 0,
  'toxic': 0},
 {'identity_hate': 0,
  'insult': 0,
  'obscene': 1,
  'severe_t